# Import packages 

In [1]:
import numpy as np 
import pylab as pl 
import geopandas as gp 
import glob 
import pandas as pd 
import sys 
sys.path.insert(0, '/home/nicolas/hpchome/network_conditioning/codes/')
import network_functions as netf

# Get the extra links that belong to Iowa 

## Write a list of the links that only intersect the area of interest

In [2]:
links_in_i = pd.read_csv('../local_data/links_in_iowa.csv', usecols=[0])
links_in_int_i = pd.read_csv('../local_data/links_in_iowa_and_touching.csv', usecols=[0])
c = np.setdiff1d(links_in_int_i.LINKNO.values, links_in_i.LINKNO.values)

In [23]:
f = open('../local_data/links_touching.csv','w')
f.write('LINKNO,\n')
for i in c:
    f.write('%d\n' % i)
f.close()

## Get a clean version of the network without the ghost links 

In [29]:
n = pd.read_csv('../local_data/links_all.csv')

In [31]:
n = netf.binary2multiple_parents(n)

In [35]:
n.to_csv('../local_data/links_all_no_ghosts.csv')

## Get the links that belong to the links over the intersection line 

We will write the task file to perform this operation. All the reuslts are saved at:
`local_data/subwatersheds/`

In [70]:
f = open('../codes/task_dempre_subwatersheds','w')
path_net = '/Users/nicolas/network_conditioning/local_data/links_all_no_ghosts.csv'
path_out = '/Users/nicolas/network_conditioning/local_data/subwatersheds/'
for i in c:
    file_out = '%s%d.csv' % (path_out, i)  
    f.write('python3 /Users/nicolas/network_conditioning/codes/dempre_upstream_links.py %s %d %s\n' % (path_net, i, file_out))
f.close()

Next, we use the generated tables to get one big table with all the links that belong to iowa, finally.

In [ ]:
#To record the advance of the algorithm
advance = np.arange(0,110,5)
cur_adv = 0

# Merge all the subwatersheds
list_subwat = glob.glob('../local_data/subwatersheds/*')
master = []
for c, i in enumerate(list_subwat):
    a = pd.read_csv(i, usecols=[1])
    master.extend(a['LINKNO'].values.astype(int))
    #master = np.unique(np.concatenate(master,a['LINKNO'].values.astype(int)))
    progress = (c / len(list_subwat))*100
    if progress > advance[cur_adv]:
        print(progress)
        cur_adv +=1

Correction due to links that do not touch the iowa border

In [3]:
inside = pd.read_csv('../local_data/links_in_iowa.csv', usecols=[0])['LINKNO'].values
noghost = pd.read_csv('../local_data/links_all_no_ghosts.csv', usecols=[1])['LINKNO'].values
inside_noghost = np.intersect1d(inside, noghost)

In [19]:
#inside = pd.read_csv('../local_data/links_in_iowa.csv', usecols=[0])['LINKNO'].values
master = master + inside_noghost.tolist()
master = np.unique(np.array(master))

List with the final selection of the network elements that belong to the region of interest 

In [21]:
f = open('../local_data/links_final_sel.csv','w')
f.write('LINKNO\n')
for lid in master:
    f.write('%d\n' % lid)
f.close()